# Etude de l'ARCEP

Nos questions : 

## Rapprochement
- combien de lignes immeubles dans le fichier de l'ARCEP ?
- combien de lignes tombent sur un bâtiment RNB ?

## Enrichissement lien bati-adresses RNB
- en prenant un échantillon de match, peut-on améliorer le lien bati ↔ adresse en se basant sur les adresses fournies par l’arcep ?
- combien de points ARCEP tombent sur des bâtiment RNB sans adresse ?

## Création de bâtiments RNB
- en prenant un échantillon de X (50?) point ARCEP tombant loin d’un bâtiment RNB : 
  - combien devraient donner lieu à la création automatique d’un bâtiment ?
  - combien sont en doute ?
    - est-ce qu’il y a un champs qui les distingue ?
  - combien sont des erreurs ?

In [11]:
import os
import csv
from pprint import pprint
import pandas as pd
from IPython.display import HTML, display
from datetime import date, datetime
import random

from batid.services.closest_bdg import get_closest_from_point
from batid.models import Building
from batid.services.guess_bdg_new import Guesser, ClosestFromPointHandler

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
data_path = "arcep_immeuble_t4_2024.csv"
guesses_path = "guesses.json"

dlpi_path = './arcep_dlpi_savoie_valdoise.csv'
dlpi_guesses_path = "dlpi_guesses.json"

**Question : combien de lignes immeubles dans le fichier de l'ARCEP ?**

In [2]:
with open(data_path, "r") as f:
    row_count = sum(1 for row in f)

    print(row_count)

24762753


**Question : combien de lignes tombent sur un bâtiment RNB ?**

In [4]:
def row_to_input(row):
    
    address = f"{row['num_voie']} {row['type_voie']} {row['nom_voie']} {row['code_poste']} {row['nom_com']}"
    
    return {
        "ext_id": f"{row['imb_id']}",
        "lat": float(row["y"]),
        "lng": float(row["x"]),
        "address": address
    }
    

On prend un échantillon de 500 000 lignes

In [21]:
with open(data_path, "r") as f:
    reader = csv.DictReader(f)

    inputs = []
    
    ext_ids = set()
    
    c = 0
    for row in reader:
        c += 1
        
        if c == 1:
            pprint(row)
        
        if row['imb_id'] not in ext_ids:
        
            inputs.append(row_to_input(row))
            ext_ids.add(row['imb_id'])

        if c > 500000:
            break

{'batiment': '',
 'catg_loc_imb': 'individuel',
 'code_insee': '57669',
 'code_l331': 'FAFO',
 'code_poste': '57450',
 'cp_no_voie': '',
 'date_completude': '',
 'date_completude_manquante': 't',
 'geom_mod': 'f',
 'imb_etat': 'deploye',
 'imb_id': 'THED1000260',
 'nom_com': 'Théding',
 'nom_voie': 'D EBRING',
 'num_voie': '35',
 'pm_etat': 'deploye',
 'pm_ref': 'N057CF2_S034',
 'type_imb': 'PA',
 'type_voie': 'RUE',
 'x': '6.900318335855359',
 'y': '49.12606398675791'}


In [22]:
g = Guesser(batch_size=2000)
g.handlers = [ClosestFromPointHandler()]
g.create_work_file(inputs,guesses_path)

In [23]:
g.guess_work_file(guesses_path)

  0%|          | 0/250 [00:00<?, ?it/s]

In [24]:
g.report()

-- Report --
Number of rows: 499971
Number of match: 457172 (91.44%)

-- finished_steps --
Rows with finished_steps closest_from_point: 499971 (100.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
point_on_bdg            440076
isolated_closest_bdg     17096
Name: count, dtype: int64

-- match_reasons : % --
match_reason
point_on_bdg            88.020305
isolated_closest_bdg     3.419398
Name: count, dtype: float64

-- Inputs --


Sur les 500000 lignes de l'échantillon, on arrive à associer 457172 identifiants RNB (91,44%).
En extrapolant aux 24762753 lignes du fichier **on peut probablement arriver à fournir 22 643 061 identifiants.**


**Question : en prenant un échantillon de match, peut-on améliorer le lien bati ↔ adresse en se basant sur les adresses fournies par l’arcep ?**

In [21]:
# We forgot to add addresses to the inputs. :(
# We add them

g = Guesser()
g.load_work_file(guesses_path)

with open(data_path, "r") as f:
    reader = csv.DictReader(f)
    
    c = 0
    for row in reader:
        
        c += 1
        if c % 200_000 == 0:
            print(c)
        
        imb_id = row['imb_id']
        address = f"{row['num_voie']} {row['type_voie']} {row['nom_voie']} {row['code_poste']} {row['nom_com']}"
        
        
        if g.guesses.get(imb_id):
            g.guesses[imb_id]["input"]["address"] = address
        
    
    g.save_work_file(guesses_path)



200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
4600000
4800000
5000000
5200000
5400000
5600000
5800000
6000000
6200000
6400000
6600000
6800000
7000000
7200000
7400000
7600000
7800000
8000000
8200000
8400000
8600000
8800000
9000000
9200000
9400000
9600000
9800000
10000000
10200000
10400000
10600000
10800000
11000000
11200000
11400000
11600000
11800000
12000000
12200000
12400000
12600000
12800000
13000000
13200000
13400000
13600000
13800000
14000000
14200000
14400000
14600000
14800000
15000000
15200000
15400000
15600000
15800000
16000000
16200000
16400000
16600000
16800000
17000000
17200000
17400000
17600000
17800000
18000000
18200000
18400000
18600000
18800000
19000000
19200000
19400000
19600000
19800000
20000000
20200000
20400000
20600000
20800000
21000000
21200000
21400000
21600000
21800000
22000000
22200000
22400000
22600000
22800000
23000000
23200000
23400000


In [31]:
# Get a random set of matches with their address
g = Guesser()
g.load_work_file(guesses_path)

import random

set_size = 30
random_matches = []
for ext_id, guess in g.guesses.items():
    
    if len(guess['matches']) > 0:
    
        # one chance on 50
        if random.randint(1, 10000) == 42:
            random_matches.append({
                'imb_id': ext_id,
                'rnb_id': guess["matches"][0],
                'address': guess["input"]["address"]
            })

            if len(random_matches) >= set_size:
                break
            

In [32]:
import pandas as pd
from IPython.display import HTML, display

df = pd.DataFrame(random_matches)

display(HTML(df.to_html()))



,imb_id,rnb_id,address
0,IMB/66037/X/05M1,7P2JBB2YF7XD,0 Avenue des Anneaux du Roussillon 66140 Canet-en-Roussillon
1,IMB/01264/X/0096,KYKNVBB57SA3,544 Route de Trévoux 01310 Montracol
2,IMB/01301/X/00BT,TA8FS8HMZQPT,176 Rue de la Croix Rouge 01310 Polliat
3,IMB/02371/X/009B,63DEQZ9C65CR,63 Rue de Bellevue 02100 Harly
4,IMB/02691/S/09PO,NQMAMDPH4DMH,35 Boulevard du Docteur Camille Guérin 02100 Saint-Quentin
5,IMB/02691/S/0CDV,4YE6B5SFERD6,94 Rue du Sentier 02100 Saint-Quentin
6,IMB/02691/X/0FOS,TJK8E7FCJG89,34 Rue Verlaine 02100 Saint-Quentin
7,IMB/03023/X/01HD,NPVWV94GYXJ6,26 Chemin de la Varenne du Léry 03700 Bellerive-sur-Allier
8,IMB/02722/X/04RU,NWSJJV5ZC19V,27 Avenue de Laon 02200 Soissons
9,IMB/03009/X/002M,7K3ZT3YGHD6T,7 Route de Bagneux 03460 Aubigny


J'ai passé en revue les adresses proposées par l'ARCEP [sur Notion](https://www.notion.so/referentielnationaldesbatiments/Etudier-int-r-t-ARCEP-14d43ec9d11e8079baf7fed793e84493?pvs=4).

- sur 30 cas, l'adresse RNB et l'adresse ARCEP diffèrent 5 fois
- pour ces 5 cas : 
  - le RNB a raison 3 fois
  - pour les 2 cas restants:
    - le RNB n'avait pas d'adresse associée à ces bâtiments
    - l'Arcep avait tord une fois et raison une fois
    
    
La conclusion est que les adresses RNB et Arcep sont très similaires (25/30). Il peut arrive que l'ARCEP propose des adresses quand le RNB n'en a aucune.

On va mesurer quel volument cela représente.

**Question : combien de points ARCEP tombent sur des bâtiment RNB sans adresse ?**

In [42]:
g = Guesser()
g.load_work_file(guesses_path)

import random

def report(matches_count, wo_addresses_count):
    
    wo_addresses_rate = wo_addresses_count / matches_count
    estimate = 24_000_000 * 0.91 * wo_addresses_rate
    
    print('---')
    print(f"RNB ID marchés : {matches_count}")
    print(f"RNB ID matchés mais sans adresses : {wo_addresses_count}")
    print("-")
    print(f"Estimation nationale de match sur batiment RNB sans adresse : {estimate}")

def count_wo_addresses(rnb_ids: set):
    
    return Building.objects.filter(
        rnb_id__in=rnb_ids,
        addresses_read_only=None
    ).count()
    

c = 0
matches = set()
matches_count = 0
wo_addresses_count = 0
for _, guess in g.guesses.items():
    
    c += 1
    if c % 10_000 == 0:
        print(f"row {c}")
    
    if matches_count > 100_000:
        break
    
    if len(guess['matches']) > 0:
        matches_count += 1
        matches.add(guess['matches'][0])
    
    if len(matches) > 1000:
        wo_addresses_count += count_wo_addresses(matches)
        matches = set()
        
        report(matches_count, wo_addresses_count)
        
        
        
wo_addresses_count += count_wo_addresses(matches)
report(matches_count, wo_addresses_count)




---
RNB ID marchés : 1015
RNB ID matchés mais sans adresses : 18
-
Estimation nationale de match sur batiment RNB sans adresse : 387310.34482758626
---
RNB ID marchés : 2046
RNB ID matchés mais sans adresses : 43
-
Estimation nationale de match sur batiment RNB sans adresse : 459002.9325513196
---
RNB ID marchés : 3108
RNB ID matchés mais sans adresses : 87
-
Estimation nationale de match sur batiment RNB sans adresse : 611351.3513513514
---
RNB ID marchés : 4190
RNB ID matchés mais sans adresses : 119
-
Estimation nationale de match sur batiment RNB sans adresse : 620276.8496420048
---
RNB ID marchés : 5279
RNB ID matchés mais sans adresses : 165
-
Estimation nationale de match sur batiment RNB sans adresse : 682629.2858495926
---
RNB ID marchés : 6282
RNB ID matchés mais sans adresses : 227
-
Estimation nationale de match sur batiment RNB sans adresse : 789188.1566380134
---
RNB ID marchés : 7290
RNB ID matchés mais sans adresses : 260
-
Estimation nationale de match sur batiment RNB

---
RNB ID marchés : 59856
RNB ID matchés mais sans adresses : 2567
-
Estimation nationale de match sur batiment RNB sans adresse : 936635.9262229351
---
RNB ID marchés : 60952
RNB ID matchés mais sans adresses : 2605
-
Estimation nationale de match sur batiment RNB sans adresse : 933409.896311852
---
RNB ID marchés : 62044
RNB ID matchés mais sans adresses : 2610
-
Estimation nationale de match sur batiment RNB sans adresse : 918741.5382631681
---
RNB ID marchés : 63082
RNB ID matchés mais sans adresses : 2621
-
Estimation nationale de match sur batiment RNB sans adresse : 907432.2310643289
---
RNB ID marchés : 64207
RNB ID matchés mais sans adresses : 2631
-
Estimation nationale de match sur batiment RNB sans adresse : 894934.1972059121
row 70000
---
RNB ID marchés : 65303
RNB ID matchés mais sans adresses : 2639
-
Estimation nationale de match sur batiment RNB sans adresse : 882589.7738235609
---
RNB ID marchés : 66471
RNB ID matchés mais sans adresses : 2657
-
Estimation nationale 

Les adresses Arcep sont assez précises mais contiennent parfois des erreurs. On pourrait les utiliser pour essayer d'obtenir les adresses des bâtiments qui n'en ont pas.

Le calcul au dessu dit que sur 100000 matches Arcep <> RNB, environ 3,3% sont sur des bâtiments RNB n'ayant aucune adresse.

Si on extrapole ça au national, on peut espérer qu'environ 700 000 bâtiments sans adresses pourrait se voir suggérés une adresse Arcep

**Question : en prenant un échantillon de X (50?) point ARCEP tombant loin d’un bâtiment RNB, combien devraient donner lieu à la création d'un bâtiment, combien en doute, combien en erreur ?**

In [53]:
g = Guesser()
g.load_work_file(guesses_path)

import random
from batid.services.closest_bdg import get_closest_from_point

set_size = 50
random_no_matches = []
for ext_id, guess in g.guesses.items():
    
    if len(guess['matches']) == 0:
    
        # one chance on 50
        if random.randint(1, 100) == 42:
            
            closest_bdg = get_closest_from_point(guess['input']['lat'], guess['input']['lng'], 25)
            
            if closest_bdg is None or (closest_bdg and closest_bdg.first().distance.m > 5):

                dist = closest_bdg.first().distance.m if closest_bdg else None
                
                random_no_matches.append({
                    'imb_id': ext_id,
                    'address': guess["input"]["address"],
                    'lat_lng': f"{guess['input']['lat']}, {guess['input']['lng']}",
                    "closest_bdg_distance": dist
                })

                if len(random_no_matches) >= set_size:
                    break

df = pd.DataFrame(random_no_matches)

display(HTML(df.to_html()))

,imb_id,address,lat_lng,closest_bdg_distance
0,ALST1000297,11 RUE SAINT PIERRE 57515 Alsting,"49.18119218995014, 6.99683338562015",17.827894
1,ALST1000796,95 RUE DE CHATELAILLON 57515 Alsting,"49.17326627172999, 7.010486285471641",7.247588
2,IMB/64125/X/01QD,3 Avenue Larraldia 64210 Bidart,"43.44019631749917, -1.5769716452513003",7.220405
3,IMB/65440/X/029E,22 Avenue Fould 65000 Tarbes,"43.22513598134338, 0.07132540202549853",14.354747
4,IMB/67131/X/01K7,0 Rue des Fusiliers Marins 67114 Eschau,"48.486768052835224, 7.730855204392565",5.673895
5,BEHR1001134,6 RUE DES CEVENNES 57460 Behren-lès-Forbach,"49.176598871643115, 6.93021565246976",8.763322
6,IMB/78396/X/018O,1 Allée des Chênes 78600 Le Mesnil-le-Roi,"48.93335667902568, 2.124675909876477",6.100471
7,ETZL1000477,6 RUE DES EGLANTIERS 57460 Etzling,"49.18326549972542, 6.959435841752154",18.935590
8,IMB/02722/X/05ML,14 Rue Jacques Brel 02200 Soissons,"49.366230094422015, 3.3130913329720038",24.730648
9,IMB/97409/X/05NG,669 Ruelle Vavangue 97440 Saint-André,"-20.95141500364648, 55.679971218684415",16.080023


Sur [46 lignes étudiées](https://www.notion.so/Etudier-int-r-t-ARCEP-14d43ec9d11e8079baf7fed793e84493?pvs=4#1d843ec9d11e80129f1dd15039fc7952) :
  - 8 batiments à créer
  - 11 points ambigus
  - 27 points où il ne faut pas créer de bâtiments

**On constate qu’on trouve très facilements des points Arcep au niveau de parcelles vides, sans doute prêtes à être construites mais n’accueillant pas de bâtiments. Les données disponibles en open data ne permettent donc pas de créer automatiquement un bâtiment, ni de detecter les fins de chantier**

**Question : en utilisant la colonne  Date Livraison Prévisionnelle Immeuble (DLPI, non open data), peut-on détecter de nouveaux bâtiments ?**

In [14]:


# build a set
kept_rows = []
old_rows = []
future_rows = []

with open(dlpi_path, "r") as f:
    reader = csv.DictReader(f)
    
    until = date(2025, 4, 18) # date I made this study
    since = date(2022, 4, 18) # more then three years ago, the BD Topo should have given us the building
    
    
    for row in reader:
        row_date = datetime.strptime(row['date_prev_livraison_imb_neuf'], "%Y-%m-%d").date()
        
        if row_date > until:
            future_rows.append(row)
            continue
            
        if row_date < since:
            old_rows.append(row)
            continue
            
        kept_rows.append(row)
        
    print(f"The file has {len(kept_rows)} in the desired time range > we want to study them")
    print(f"{len(old_rows)} old rows > too old")
    print(f"{len(future_rows)} future rows > interesting for later")
            
        


The file has 3414 in the desired time range > we want to study them
3347 old rows > too old
609 future rows > interesting for later


In [19]:
dlpi_dict = {row['code_imb']: row['date_prev_livraison_imb_neuf'] for row in kept_rows}

print(dlpi_dict)

{'IMB/73213/X/01SR': '2025-03-03', 'IMB/73008/X/05S8': '2023-12-31', 'IMB/73008/X/05SW': '2023-12-31', 'IMB/73008/X/05VR': '2024-06-24', 'IMB/73008/X/05VS': '2024-06-24', 'IMB/73008/X/05VT': '2024-06-24', 'IMB/73008/X/05WL': '2022-11-01', 'IMB/73008/X/05Y3': '2022-11-01', 'IMB/73008/X/05Y4': '2022-11-01', 'IMB/73008/X/05Y5': '2022-11-01', 'IMB/73008/X/05YO': '2023-02-28', 'IMB/73008/X/05ZO': '2025-02-17', 'IMB/73008/X/060D': '2022-11-15', 'IMB/73008/X/060E': '2022-11-15', 'IMB/73008/X/060F': '2022-11-15', 'IMB/73008/X/060G': '2022-11-15', 'IMB/73008/X/060H': '2023-01-13', 'IMB/73008/X/060I': '2023-01-13', 'IMB/73008/X/060J': '2023-01-13', 'IMB/73008/X/060S': '2023-06-30', 'IMB/73008/X/060T': '2023-09-01', 'IMB/73008/X/060U': '2023-09-01', 'IMB/73008/X/060V': '2022-11-07', 'IMB/73008/X/060W': '2022-11-07', 'IMB/73008/X/061A': '2023-12-31', 'IMB/73008/X/061C': '2023-07-15', 'IMB/73008/X/061D': '2024-03-11', 'IMB/73008/X/061E': '2024-03-11', 'IMB/73008/X/061F': '2024-03-11', 'IMB/73008/X/

In [20]:
# make a set of imb_id we look for
dlpi_imb_id = {row['code_imb'] for row in kept_rows}
already_found_imb_ids = set()



inputs = []
with open(data_path, "r") as f:
    reader = csv.DictReader(f)
    
    for row in reader:
        
        if row['imb_id'] in dlpi_imb_id and row['imb_id'] not in already_found_imb_ids:
            

            
            guess_input = row_to_input(row)
            guess_input['dlpi'] = dlpi_dict.get(row['imb_id'], None)
            inputs.append(guess_input)
            
            already_found_imb_ids.add(row['imb_id'])
            

    
g = Guesser()
g.create_work_file(inputs,dlpi_guesses_path)
            
print(f"going to guess {len(g.guesses)} rows")       


going to guess 3414 rows


In [21]:
g = Guesser()
g = Guesser(batch_size=500)
g.handlers = [ClosestFromPointHandler()]

g.guess_work_file(dlpi_guesses_path)


  0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
g.report()

-- Report --
Number of rows: 3414
Number of match: 1573 (46.07%)

-- finished_steps --
Rows with finished_steps closest_from_point: 3414 (100.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
point_on_bdg            1322
isolated_closest_bdg     251
Name: count, dtype: int64

-- match_reasons : % --
match_reason
point_on_bdg            38.722906
isolated_closest_bdg     7.352080
Name: count, dtype: float64

-- Inputs --


In [24]:
g = Guesser()
g.load_work_file(dlpi_guesses_path)



set_size = 50
random_no_matches = []
for ext_id, guess in g.guesses.items():
    
    if len(guess['matches']) == 0:
    
        if random.randint(1, 10) == 7:
            
            closest_bdg = get_closest_from_point(guess['input']['lat'], guess['input']['lng'], 25)
            
            if closest_bdg is None or (closest_bdg and closest_bdg.first().distance.m > 5):

                dist = closest_bdg.first().distance.m if closest_bdg else None
                
                random_no_matches.append({
                    'imb_id': ext_id,
                    'address': guess["input"]["address"],
                    'lat_lng': f"{guess['input']['lat']}, {guess['input']['lng']}",
                    "dlpi": guess["input"]["dlpi"],
                    "closest_bdg_distance": dist
                })

                if len(random_no_matches) >= set_size:
                    break

df = pd.DataFrame(random_no_matches)

display(HTML(df.to_html()))

,imb_id,address,lat_lng,dlpi,closest_bdg_distance
0,IMB/73011/X/038X,468 Chemin de la Cassine 73200 Albertville,"45.65985977571762, 6.377008624502703",2024-03-15,17.540185
1,IMB/73011/X/038Z,468 Chemin de la Cassine 73200 Albertville,"45.65948557200186, 6.377314352225055",2024-03-15,15.222767
2,IMB/73029/X/00Z4,6 Chemin des Prés 73000 Barberaz,"45.5580079914113, 5.947006863383577",2023-06-26,19.482402
3,IMB/73030/X/00UO,224 Avenue Principale 73230 Barby,"45.57014789737111, 5.978032795817263",2024-09-27,7.470579
4,IMB/73031/X/00V0,676 Chemin des Monts Dessus 73000 Bassens,"45.58060053299912, 5.9324638239279395",2025-04-01,14.794603
5,IMB/73051/X/01GR,39 Chemin du Nivolet 73370 Le Bourget-du-Lac,"45.62339539231981, 5.850904584665455",2023-03-07,19.363715
6,IMB/73051/X/01HJ,0 le Petit Caton 73370 Le Bourget-du-Lac,"45.65894898078106, 5.851183526156194",2023-07-25,20.839923
7,IMB/73051/X/01HU,1451 Route des Catons 73370 Le Bourget-du-Lac,"45.64095836023014, 5.849435719579955",2024-10-08,20.200779
8,IMB/73008/X/068S,110 Avenue de Saint-Simond 73100 Aix-les-Bains,"45.70342536459243, 5.913185722783526",2025-04-15,15.806657
9,IMB/73182/X/00WV,12 Impasse du Hameau du Panoramic 73100 Mouxy,"45.68000041831455, 5.920004392905788",2023-12-31,15.927407


In [31]:
savoie = 302946
valdoise = 378818

extract_real_bdgs = savoie + valdoise
total_real_bdgs = 44056425

extract_ratio = extract_real_bdgs / total_real_bdgs
print(extract_ratio)

total_dlpi = 3414
dlpi_success_rate = 0.85

dlpi_success_nat = (total_dlpi * dlpi_success_rate) / extract_ratio

print(f"Nombre de nouveaux batiments indiqués grâce aux DLPI : {dlpi_success_nat}")

0.015474791701777891
Nombre de nouveaux batiments indiqués grâce aux DLPI : 187524.33350470252
